In [1]:
! nvidia-smi

Fri Aug 11 01:32:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  Off |
|  0%   37C    P5    58W / 375W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%pip install ruprompts
!pip install accelerate -U
!pip install datasets

Note: you may need to restart the kernel to use updated packages.


# ruPROMPTs tutorial

This [tutorial](https://github.com/ai-forever/ru-prompts/blob/main/notebooks/detox-russe-train-python.ipynb) presents an example of prompt-tuning with ruPROMTS framework for the detoxification task.

In [2]:
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
!wget https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/test.tsv

--2023-08-11 01:32:21--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1902888 (1.8M) [text/plain]
Saving to: ‘train.tsv.2’

train.tsv.2         100%[===================>]   1.81M  --.-KB/s    in 0.05s   

2023-08-11 01:32:21 (34.3 MB/s) - ‘train.tsv.2’ saved [1902888/1902888]

--2023-08-11 01:32:21--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2006

## Imports

In [3]:
import os

import pandas as pd
import shutil
from tqdm import tqdm

from datasets import load_dataset

import transformers
from transformers import GPT2LMHeadModel, AutoTokenizer
from transformers import set_seed
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from transformers import Trainer
from transformers import TrainingArguments
from transformers import pipeline


from ruprompts import TensorPromptProvider, LSTMPromptProvider
from ruprompts import Text2TextPreprocessor
from ruprompts import Prompt, PromptFormat

from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Data

In [4]:
# df = pd.read_csv("train.tsv", sep="\t")
# df.drop(["index"], axis=1, inplace=True)
# df.to_csv("train.tsv", index=False, sep="\t")

datasets = load_dataset("csv", 
                        data_files={"train": "train.tsv", 
                                    "validation": "dev.tsv",
                                   }, sep="\t")
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]
test_data = pd.read_csv("test.tsv", sep="\t")['toxic_comment'].tolist()

## Model

In [5]:
backbone_id = "sberbank-ai/rugpt3large_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Training TensorPromptProvider

In [6]:
set_seed(1)

prompt_format = PromptFormat("<P*100>{toxic_comment}<P*20>")
prompt_provider = TensorPromptProvider()

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="neutral_comment1",
    max_tokens=1792,
    truncation_field="toxic_comment",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

training_args = TrainingArguments(
    output_dir="./TensorPromptProvider",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_steps=1000,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=100000,
    #report_to="tensorboard",
    # report_to=["tensorboard", "wandb"],  # uncomment to log to WandB
    logging_dir="logs",
    seed=1,
)

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=2000,
    num_training_steps=training_args.max_steps,
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1000,2.152600,2.034594
2000,1.950200,1.764478
3000,1.828900,1.718937
4000,1.616300,1.723563
5000,1.650200,1.549067
6000,1.626400,1.565487
7000,1.635800,1.517514
8000,1.549000,1.618584
9000,1.513400,1.547968
10000,1.368500,1.480748


TrainOutput(global_step=100000, training_loss=1.2705454140472412, metrics={'train_runtime': 5635.404, 'train_samples_per_second': 35.49, 'train_steps_per_second': 17.745, 'total_flos': 1.3368979771224883e+17, 'train_loss': 1.2705454140472412, 'epoch': 28.79})

## Training LSTMPromptProvider

In [7]:
set_seed(1)

prompt_format = PromptFormat("<P*100>{toxic_comment}<P*20>")
prompt_provider = LSTMPromptProvider()

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="neutral_comment1",
    max_tokens=1792,
    truncation_field="toxic_comment",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

training_args = TrainingArguments(
    output_dir="./LSTMPromptProvider",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_steps=1000,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.01,
    max_steps=100000,
    #report_to="tensorboard",
    # report_to=["tensorboard", "wandb"],  # uncomment to log to WandB
    logging_dir="logs",
    seed=1,
)

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=2000,
    num_training_steps=training_args.max_steps,
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1000,3.000800,3.308973
2000,3.256000,3.242140
3000,3.118900,3.172971
4000,3.116200,3.085704
5000,3.026600,2.970207
6000,3.114900,2.932387
7000,2.991300,2.886102
8000,3.040400,2.868218
9000,3.009400,2.883643
10000,2.771800,2.889516


KeyboardInterrupt: 

## Inference

In [24]:
best_checkpoint = './TensorPromptProvider/checkpoint-52000'

prompt = Prompt.from_pretrained(best_checkpoint)

ppln = pipeline("text2text-generation-with-prompt", 
                prompt=prompt, 
                model=model, 
                tokenizer=tokenizer, 
                device=0)

ppln({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"}, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'А ну иди сюда'}]

In [14]:
all_ = []
top_p_predictions = []

beam_count = 3

for idx, i in enumerate(tqdm(valid_dataset["toxic_comment"])):
    options = ppln(
        {"toxic_comment": i},
        do_sample=True,
        num_beams=beam_count,
        num_return_sequences=beam_count,
        top_k=50,
        top_p=0.95,
        early_stopping=True,
    )
    
    #print(options)
    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    top_p_predictions.append(answer)

  9%|▉         | 70/800 [00:16<02:24,  5.07it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 24%|██▍       | 193/800 [00:46<02:05,  4.85it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
 26%|██▋       | 211/800 [00:50<02:21,  4.16it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 44%|████▍     | 351/800 [01:27<02:06,  3.55it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 61%|██████▏   | 491/800 [02:02<01:07,  4.56it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 64%|██████▎   | 508/800 [02:06<01:10,  4.12it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
 70%|███████   | 562/800 [02:19<00:51,  4.62it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 88%|████████▊ | 702/800 [02:55<00:39,  2.49it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


100%|██████████| 800/800 [03:20<00:00,  3.98it/s]


In [15]:
top_p_predictions[:3]

['Температуры горения хватит чтобы её расплавить',
 'А ты там был. Ты вообще служил',
 'а сам где кормишься?']